In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#cria o servico pro chrome 
servico = Service(ChromeDriverManager().install())
#inicializa o chorme
#navegador = webdriver.Chrome(service=servico)
#abre a pagina que precisa
#navegador.get("https://salaryaftertax.com/ie/salary-calculator")

In [ ]:
#abre a pagina que precisa
import time
from selenium.webdriver.common.by import By

##start the chorme\ inicializa o chorme
navegador = webdriver.Chrome(service=servico)
navegador.get("https://salaryaftertax.com/ie/salary-calculator")
#this 2 needs to be together\esses dois precisam estar juntos^^
print("Wait for it...")
time.sleep(2)  # Wait for 2 seconds
print("Done waiting!")


#sometiems appears to accept the conditions so it searchs for it if doesn't find it goes to next option
try:
    navegador.find_element(By.ID, 'ez-accept-all').click()
except Exception as error:
    pass

#Click in the element to choose the option "weekly"\clica no elemento para achar a opção "weekly"(semanalmente)
navegador.find_element(By.ID, 'payPeriod').click()
navegador.find_element(By.XPATH, '//*[@id="payPeriod"]/option[4]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="salary-calculator-v4-form"]/button').click()

#create the list for the net income and gross income\ cria a lista para o salario liquido e salario bruto
net_income = []
gross_income = []

#starts the loop to get the range of salary\ cria o loop para que pegue o loop dos salarios
for i in range(260,999):
    
    #eliminates whats written in the camp "gross salary"\exclui o que estiver escrito
    time.sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="grossSalary"]').clear()
    time.sleep(1)
    #writes the value of the gross pay\escreve no caminho a valor do salrio bruto
    navegador.find_element(By.XPATH, '//*[@id="grossSalary"]').send_keys(i)
   
    #click in the button to calculate the net pay\clica no botao pra calcular o salario liquido
    navegador.find_element(By.XPATH, '//*[@id="salary-calculator-v4-form"]/button').click()
    
    #wait to see if the page loaded ot get the number\ confere se a pagina carregou pra pegar o valor
    condition_met = False
    while not condition_met:
        try:
            element_text = navegador.find_element(By.XPATH, '//*[@id="tax-breakdown-table"]/tbody/tr[1]/td[3]/span').text
            condition_met = (element_text[1:] == str(i))
            if not condition_met:
                time.sleep(0.5)
        except (NoSuchElementException, StaleElementReferenceException):
            time.sleep(0.5)
            continue
    
    #get the net salary\pega o valor do net_pay
    valor = navegador.find_element(By.XPATH, '//*[@id="tax-breakdown-table"]/tbody/tr[7]/td[3]/span').text
    
    #add the values to the lists\ adiciona os valores nas listas
    net_income.append(int(valor[1:]))
    gross_income.append(i)

print(gross_income)
print(net_income)


In [10]:
import pandas as pd
import os

# Define the data as dictonary\define os valores como dicionario
data = {
    "Gross Pay": gross_income,
    "Net Pay": net_income
}

# Create a DataFrame from the data dictionary\cria a tabela com o dicionario
df = pd.DataFrame(data)

# Define the path to save the Excel file\define o caminho do excel
excel_file = "salary_data.xlsx"

# Write the DataFrame to an Excel file\
df.to_excel(excel_file, index=False)

print("Excel file created successfully.")


# Check if the file exists before attempting to open it
if os.path.exists(excel_file):
    # Open the Excel file
    os.system(f"start excel {excel_file}")
else:
    print("Excel file not found.")

Excel file created successfully.
